In [ ]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
# feature scaling
from sklearn.preprocessing import StandardScaler

# for tree binarisation
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV


# to build the models
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [ ]:
train = pd.read_csv('../input/beyond-analysis/train.csv')
test = pd.read_csv('../input/beyond-analysis/test.csv')

In [ ]:
train.shape

In [ ]:
y = train['CATEGORY_1'].unique()
y1 = test['CATEGORY_1'].unique()
a = []
for i in y:
    a.append(i)
for i in y1:
    a.append(i)
encoder = LabelEncoder()
encoder.fit(a)
train['cat1'] = encoder.transform(train['CATEGORY_1'])
test['cat1'] = encoder.transform(test['CATEGORY_1'])

In [ ]:
y = train['CATEGORY_2'].unique()
y1 = test['CATEGORY_2'].unique()
a = []
for i in y:
    a.append(i)
for i in y1:
    a.append(i)
encoder1 = LabelEncoder()
encoder1.fit(a)
train['cat2'] = encoder1.transform(train['CATEGORY_2'])
test['cat2'] = encoder1.transform(test['CATEGORY_2'])

In [ ]:
train.drop(['CATEGORY_1', 'CATEGORY_2'], axis=1, inplace=True)
test.drop(['CATEGORY_1', 'CATEGORY_2'], axis=1, inplace=True)

In [ ]:
a = {}
for i in train.columns:
    if(i=='UNIQUE_IDENTIFIER'):
        continue
    a[str(i)] = 'mean'

In [ ]:
a1 = {}
for i in test.columns:
    if(i=='UNIQUE_IDENTIFIER'):
        continue
    a1[str(i)] = 'mean'

In [ ]:
aggregation_functions = a
train1 = train.groupby(train['UNIQUE_IDENTIFIER'], as_index=False).aggregate(aggregation_functions)
aggregation_functions = a1
test1 = test.groupby(test['UNIQUE_IDENTIFIER'], as_index=False).aggregate(aggregation_functions)

In [ ]:
train1.head()

In [ ]:
train2 = train.drop(['Y1', 'Y2', 'cat1', 'cat2', 'DEPOSIT_2', 'DEPOSIT_TRAILS'], axis=1)
test2 = test.drop(['cat1', 'cat2', 'DEPOSIT_2', 'DEPOSIT_TRAILS'], axis=1)

In [ ]:
import scipy

In [ ]:
def b(a):
    return np.quantile(a, 0.75)

In [ ]:
def b1(a):
    return np.quantile(a, 0.25)

In [ ]:
a = {}
for i in train2.columns:
    if(i=='UNIQUE_IDENTIFIER'):
        continue
    if(i=='SEQUENCE_NO'):
        a[str(i)] = [np.mean, np.max]
    else:
        a[str(i)] = [np.mean, np.max, np.min, np.std, np.median, b, b1]
aggregation_functions = a
df_train = train2.groupby(train2['UNIQUE_IDENTIFIER'], as_index=False).agg(aggregation_functions)
aggregation_functions = a
df_test = test2.groupby(test2['UNIQUE_IDENTIFIER'], as_index=False).agg(aggregation_functions)

In [ ]:
df_train.head()

In [ ]:
df_train['cat1'] = train1['cat1']
df_test['cat1'] = test1['cat1']
df_train['cat2'] = train1['cat2']
df_test['cat2'] = test1['cat2']
df_train['Y1'] = train1['Y1']
df_train['Y2'] = train1['Y2']

In [ ]:
df_train.head()

In [ ]:
target_cols = ['Y1', 'Y2']
feature_cols = [col for col in df_train.columns if col not in [('Y2', ''), ('UNIQUE_IDENTIFIER', ''), ('Y1', '')]]

In [ ]:
def train_and_evaluate(train, test):
    # Hyperparammeters (optimized)
    seed = 1111
    params = {
        'learning_rate': 0.001,        
        'lambda_l1': 2,
        'lambda_l2': 2,
        'num_leaves': 750,
        'min_sum_hessian_in_leaf': 20,
        'feature_fraction': 0.8,
        'feature_fraction_bynode': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 42,
        'min_data_in_leaf': 100,
        'max_depth': 6,
        'seed': seed,
        'feature_fraction_seed': seed,
        'bagging_seed': seed,
        'drop_seed': seed,
        'data_random_seed': seed,
        'objective': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs': -1,
    }     
    
    # Split features and target
    x = train[feature_cols]
    y = train['Y2']
    x_test = test[feature_cols]
    x = x.values
    x_test = x_test.values
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x[trn_ind], x[val_ind]
        y_train, y_val = y[trn_ind], y[val_ind]
        train_dataset = lgb.Dataset(x_train, y_train)
        val_dataset = lgb.Dataset(x_val, y_val)
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 10000, early_stopping_rounds = 1000, 
                          verbose_eval = 100)
#         model = lgb.LGBMRegressor(n_estimators=10000, random_state=47,learning_rate=0.005,importance_type = 'gain',
#                      n_jobs = -1,metric='rmse')
#         model.fit(x_train,y_train,eval_set = [(x_train,y_train),(x_val,y_val)],early_stopping_rounds=500,verbose=50)
        plt.figure(figsize=(12,6))
        lgb.plot_importance(model, max_num_features=20)
        plt.title("Feature importance")
        plt.show()
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        test_predictions += model.predict(x_test) / 5
        
    rmspe_score = sqrt(mean_squared_error(y, oof_predictions))
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

# Traing and evaluate
test_predictions1 = train_and_evaluate(df_train, df_test)

In [ ]:
def train_and_evaluate(train, test):
    # Hyperparammeters (optimized)
    seed = 1111
    params = {
        'learning_rate': 0.001,        
        'lambda_l1': 1,
        'lambda_l2': 2,
        'num_leaves': 256,
        'min_sum_hessian_in_leaf': 20,
        'feature_fraction': 0.8,
        'feature_fraction_bynode': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 42,
        'min_data_in_leaf': 100,
        'max_depth': 7,
        'seed': seed,
        'feature_fraction_seed': seed,
        'bagging_seed': seed,
        'drop_seed': seed,
        'data_random_seed': seed,
        'objective': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs': -1,
    }     
    
    # Split features and target
    x = train[feature_cols]
    y = train['Y1']
    x_test = test[feature_cols]
    x = x.values
    x_test = x_test.values
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x[trn_ind], x[val_ind]
        y_train, y_val = y[trn_ind], y[val_ind]
        train_dataset = lgb.Dataset(x_train, y_train)
        val_dataset = lgb.Dataset(x_val, y_val)
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 30000, early_stopping_rounds = 1000, 
                          verbose_eval = 100)
#         model = lgb.LGBMRegressor(n_estimators=10000, random_state=47,learning_rate=0.005,importance_type = 'gain',
#                      n_jobs = -1,metric='rmse')
#         model.fit(x_train,y_train,eval_set = [(x_train,y_train),(x_val,y_val)],early_stopping_rounds=500,verbose=50)
        plt.figure(figsize=(12,6))
        lgb.plot_importance(model, max_num_features=20)
        plt.title("Feature importance")
        plt.show()
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        test_predictions += model.predict(x_test) / 5
        
    rmspe_score = sqrt(mean_squared_error(y, oof_predictions))
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

# Traing and evaluate
test_predictions = train_and_evaluate(df_train, df_test)

In [ ]:
target_cols = ['Y1', 'Y2']
feature_cols1 = [col for col in train1.columns if col not in ['Y2', 'UNIQUE_IDENTIFIER', 'Y1', 'DEPOSIT_2', 'DEPOSIT_TRAILS']]

In [ ]:
def train_and_evaluate(train, test):
    # Split features and target
    x = train[feature_cols1]
    y = train['Y1']
    x_test = test[feature_cols1]
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        model = MLPRegressor(hidden_layer_sizes=(32,32), max_iter=2000, learning_rate='adaptive', early_stopping=True, activation='tanh', verbose=True, random_state=1)
        model.fit(x_train, y_train)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        test_predictions += model.predict(x_test) / 5
        
    rmspe_score = sqrt(mean_squared_error(y, oof_predictions))
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

# Traing and evaluate
preds1 = train_and_evaluate(train1, test1)

In [ ]:
def train_and_evaluate(train, test):
    # Split features and target
    x = train[feature_cols]
    y = train['Y2']
    x.replace([np.inf, -np.inf], np.nan, inplace=True)
    x.fillna(x.mean(), inplace=True)
    x_test = test[feature_cols]
    x_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    x_test.fillna(x_test.mean(), inplace=True)
    x = x.values
    x_test = x_test.values
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x[trn_ind], x[val_ind]
        y_train, y_val = y[trn_ind], y[val_ind]
        model = MLPRegressor(hidden_layer_sizes=(24,24,16), max_iter=2000, learning_rate='adaptive', early_stopping=True, activation='logistic', verbose=True, random_state=1)
        model.fit(x_train, y_train)
        pred = model.predict(x_train)
        print('rf train rmse: {}'.format(sqrt(mean_squared_error(y_train, pred))))
        pred = model.predict(x_val)
        print('rf test rmse: {}'.format(sqrt(mean_squared_error(y_val, pred))))
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        test_predictions += model.predict(x_test) / 5
        
    rmspe_score = sqrt(mean_squared_error(y, oof_predictions))
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

# Traing and evaluate
preds2 = train_and_evaluate(df_train, df_test)

In [ ]:
sub = pd.DataFrame({'UNIQUE_IDENTIFIER': test1['UNIQUE_IDENTIFIER'], 
                   'Y1': test_predictions*0.8+0.2*preds1, 'Y2': test_predictions1*0.83+0.17*preds2})

In [ ]:
sub.to_csv('sub8.csv', index=False)